In [1]:
import pandas as pd
import numpy as np
import json
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
team_metrics_df = pd.read_csv("team_metric_df.csv")
print(len(team_metrics_df))
team_metrics_df.head()

431091


,Unnamed: 0,Count HI_now,Count HI_half,Count HI_game,Count HSR_now,Count HSR_half,Count HSR_game,Count High Acceleration_now,Count High Acceleration_half,Count High Acceleration_game,Count High Deceleration_now,Count High Deceleration_half,Count High Deceleration_game,Count Medium Acceleration_now,Count Medium Acceleration_half,Count Medium Acceleration_game,Count Medium Deceleration_now,Count Medium Deceleration_half,Count Medium Deceleration_game,Count Sprint_now,Count Sprint_half,Count Sprint_game,High Intensity (HI) Distance_now,High Intensity (HI) Distance_half,High Intensity (HI) Distance_game,High Speed Running (HSR) Distance_now,High Speed Running (HSR) Distance_half,High Speed Running (HSR) Distance_game,M/min_now,M/min_half,M/min_game,Max Speed_now,Max Speed_half,Max Speed_game,Running Distance_now,Running Distance_half,Running Distance_game,Sprinting Distance_now,Sprinting Distance_half,Sprinting Distance_game,Total Distance_now,Total Distance_half,Total Distance_game,event_id,gameId,player_id,shooter_id_x,match_date,minute,event_index,period,timestamp,second,possession,duration,obv_for_after,obv_for_before,obv_for_net,obv_against_after,obv_against_before,obv_against_net,obv_total_net,under_pressure,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,shot_team_id,team_name,location_x,location_y,location_z,related_events,shooter_id_y,player_name,position_id,position_name,statsbomb_xg,goal,penalty,body_part,opposition_in_triangle,Count High Acceleration_1_week,Total Distance_1_week,Sprinting Distance_1_week,Running Distance_1_week,Max Speed_1_week,M/min_1_week,High Speed Running (HSR) Distance_1_week,High Intensity (HI) Distance_1_week,Count Sprint_1_week,Count Medium Deceleration_1_week,Count Medium Acceleration_1_week,Count High Deceleration_1_week,Count HSR_1_week,Count HI_1_week,nan_1_week,Count High Acceleration_2_week,Total Distance_2_week,Sprinting Distance_2_week,Running Distance_2_week,Max Speed_2_week,M/min_2_week,High Speed Running (HSR) Distance_2_week,High Intensity (HI) Distance_2_week,Count Sprint_2_week,Count Medium Deceleration_2_week,Count Medium Acceleration_2_week,Count High Deceleration_2_week,Count HSR_2_week,Count HI_2_week,nan_2_week,Count High Acceleration_1_month,Total Distance_1_month,Sprinting Distance_1_month,Running Distance_1_month,Max Speed_1_month,M/min_1_month,High Speed Running (HSR) Distance_1_month,High Intensity (HI) Distance_1_month,Count Sprint_1_month,Count Medium Deceleration_1_month,Count Medium Acceleration_1_month,Count High Deceleration_1_month,Count HSR_1_month,Count HI_1_month,nan_1_month,shooter_id,birth_date
0,0,0.6,0.6,0.6,0.6,0.6,0.6,0.8,0.8,0.8,0.8,0.8,0.8,7.2,7.2,7.2,5.6,5.6,5.6,0.0,0.0,0.0,3.051964,3.051964,3.051964,3.051964,3.051964,3.051964,21.607557,21.607557,21.607557,4.432794,4.432794,4.432794,32.360770,32.360770,32.360770,0.000000,0.000000,0.000000,324.113359,324.113359,324.113359,24392d1d-314a-4063-a729-7775bc0a2c45,3892755,3306,23616,11/08/2023,3,130,1,03:04.5,4,9,0.471596,0.306241,0.278243,0.027997,0.007197,0.007197,0.0,0.027997,True,16,Shot,36,Manchester City,2,From Corner,36,Manchester City,112.7,39.9,0.22,"67dc6041-d315-4ebb-ac8e-18f3fcd24abc, 7d4f2f91...",23616,Erling Håland,23,Center Forward,0.166126,1,0,Left Foot,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18/02/1995
1,1,1.8,1.8,1.8,1.4,1.4,1.4,1.2,1.2,1.2,0.4,0.4,0.4,5.8,5.8,5.8,5.6,5.6,5.6,0.4,0.4,0.4,23.223715,23.223715,23.223715,17.697490,17.697490,17.697490,23.007279,23.007279,23.007279,5.700157,5.700157,5.700157,45.301437,45.301437,45.301437,5.526225,5.526225,5.526225,345.109181,345.109181,345.109181,24392d1d-314a-4063-a729-7775bc0a2c45,3892755,3193,23616,11/08/2023,3,130,1,03:04.5,4,9,0.471596,0.306241,0.278243,0.027997,0.007197,0.007197,0.0,0.027997,True,16,Shot,36,Manchester City,2,From Corner,36,Manchester City

see if overall team tiredness affects shots and shot creation

group by gameId and event number, with summed metrics and individual metrics and group and indivd acwr? but first xg and over_xg